# Using Pydantic for Data Validation

Instalando biblioteca pydantic[email], pois estava dando um erro ao executar
a class Role, devido ao uso do EmailStr

In [1]:
!pip install pydantic[email]

Abaixo temos a importação de módulos e classes que ajudam na criação de modelos de dados validados (pydantic) (typing, any aceitando qualquer tipo de dado) e na definição de enumerações (enum, sendo auto e IntFlag).

In [2]:
from enum import auto, IntFlag
from typing import Any
from pydantic import (BaseModel,
                      EmailStr, # tipos já válidos da biblioteca pydantic
                      Field,
                      SecretStr,
                      ValidationError,
                      )

Abaixo temos duas classes um Role enum, a qual é um int e pode ser um author, editor, developer ou admin que pode ser qualquer um dos três. A user class que herda BaseModel, que foi importada do pydantic e é usada para realizar o básico da validação do pydantic, os campos que fazem parte da classe user, ou seja, o que o usuário deve ter são name ( o qual têm tipo de anotação str e um Field com o exemplos de nomes válidos), email ( usa o tipo EmailStr que veio da biblioteca pydantic e também possuí um Field que contêm alguns parâmetros como exemplos válidos, uma descrição sobre o que é e se deve ficar congelado sem poder ser alterado depois ou não, o "frozen"), password (O mesmo vale para a senha, a qual têm o tipo SecretStr que veio da biblioteca pydantic, o qual quando é imprimido exibe ****** e além disso password possui exemplo e descrição) e role ( que recebe os valores enumerados provenientes da classe Role).

In [3]:
class Role(IntFlag):
  Author = auto()
  Editor = auto()
  Developer = auto()
  Admin = Author | Editor | Developer

class User(BaseModel):
    name: str = Field(examples=["Lucas_Cristiano"])
    email: EmailStr = Field(
        examples=["lucas_cristiano@discente.ufg.br"],
        description="Email do discente",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Lucas4321"], description="Senha do Usuário"
    )
    role: Role = Field(default=None, description="Ocupação do Usuário") # Adicionando Role opcional
    projeto: str = Field (examples=["Fastcamp_Agentes"]) # Adcionando um projeto que a pessoa está envolvida
    perido: str = Field (examples=["2025.1"]) # Adicionando o periodo que a pessoa está no momento
    semestre: int = Field (examples=1) # Adicionando o valor numérico do semestre dessa pessoa

Abaixo temos definida uma função validação (validate), a qual chama o modelo de validação e nos permite verificar se os dados estão de acordo com a estrutura de usuário definida anteriormente. Logo se estiverem o usuário é imprimido se não imprime que o usuário é inválido e imprime os erros.

In [4]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("Usuário não é válido")
        for error in e.errors():
            print(error)

Abaixo temos um exemplo de dados corretos (good_data) e incorretos (bad_data) para testar a função validate e verificar se os dados estão de acordo com a estrutura de usuário definida em User.

In [5]:
def main() -> None:
    good_data = {
        "name": "Lucas",
        "email": "lucas_cristiano@discente.ufg.br",
        "password": "Password123",
        "projeto": "Fastcamp_Agentes",
        "perido": "2025.1",
        "semestre": 1
    }
    bad_data = {"email": "@discente", "password": "lucas4321"} # espera-se que ocorra um erro no e-mail devido a não ter nada antes do @

    validate(good_data)
    validate(bad_data)

Quando executado obtemos para o good_data tudo certo e para o bad_data recebemos um erro de que não há um name e de que ambos os valores de email e de password não são válidos. Logo abaixo temos referências da documentação para entender o erro.

In [6]:
if __name__ == "__main__":
  main()

name='Lucas' email='lucas_cristiano@discente.ufg.br' password=SecretStr('**********') role=None projeto='Fastcamp_Agentes' perido='2025.1' semestre=1
Usuário não é válido
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '@discente', 'password': 'lucas4321'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: There must be something before the @-sign.', 'input': '@discente', 'ctx': {'reason': 'There must be something before the @-sign.'}}
{'type': 'missing', 'loc': ('projeto',), 'msg': 'Field required', 'input': {'email': '@discente', 'password': 'lucas4321'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'missing', 'loc': ('perido',), 'msg': 'Field required', 'input': {'email': '@discente', 'password': 'lucas4321'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'missing', 'loc': ('semestre',), 'msg': 'Field required', 'input': {'email

Pelo visto o único erro foi no e-mail como mostra o output devido ao não uso de alguma coisa antes do @ e também vemos que como os outro fields não foram respondidos mesmo sendo obrigatórios todos apresentaram erro com exeção de Role que foi configurado como opcional.

# Custom Validators

A diferença em relação ao exemplo anterior é o módulo hashlib para a implementação de criptografia e o re para usar regex.

In [7]:
import enum
import hashlib
import re
from typing import Any, Self

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

Expressões regulares para validar nomes e senhas

In [8]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

Abaixo temos o class Role e o Class user modificados, primeiramente sobre o class Role agora ele está recebendo diretamente valores int para cada Role, em vez de auto como no código anterior. Já a class User teve modificações quanto aos objetos pydantic adicionados, os quais são validações personalizadas, as quais não estão presentes na biblioteca pydantic, usando fields validators decorators, o primero é de nome que é definido um valor de classe, onde está sendo verificado pelo regex se o valor corresponde ao obtido e se não é gerado um valor de erro de que o nome deve conter letras e pelo menos 2 caracteres, além disso também não temos um mode nele. Já no validator de field temos um "mode=before", onde pode-se especificar o tipo de entrada que se deseja trabalhar e no caso da função validate_role pode ser um valor inteiro, string ou enum e então tenta-se converter o valor pra um tipo Role e se não funcionar é emitido um erro. Além disso vale citar que ao usar o modo before essas funções de validação deverão ser metódos de classe (No caso do "mode=after", tratam-se de metódos de instância regulares, podendo só passar o self e trabalhar com ele). Logo em seguida temos objeto model_validator, que permite validar todos os ddadfos de uam vez, o qual é útil para verificar se o usuário não está usando o seu name contido no password, onde é usado um dicionário, além disso vale citar que também há o uso de regex para verificar a senha, a qual vai ser inválida se não ter 8 caracteres, 1 maiúscula, 1 minúscula e 1 número, e antes de finalizar temos um hash da senha antes de armzená-la na instância.

In [9]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Lucas_Cristiano"])
    email: EmailStr = Field(
        examples=["lucas_cristiano@discente.ufg.br"],
        description="Email do discente",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Lucas4321"], description="Senha do Usuário"
    )
    role: Role = Field(default=None, description="Ocupação do Usuário")
    projeto: str = Field (examples=["Fastcamp_Agentes"])
    periodo: str = Field (examples=["2025.1"])
    semestre: int = Field (examples=1)

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @field_validator("semestre",mode="before") # se semestre for um valor negativo ou maior que dois aparecerá um erro
    @classmethod
    def validate_semestre(cls, v:int) -> int:
        if v < 0:
            raise ValueError("Semestre não pode ser negativo")
        elif v > 2:
            raise ValueError("Semestre não pode ser maior que 2")

        return v

    @field_validator("projeto",mode="before") # se projeto for vazio irá acontecer um erro
    @classmethod
    def validate_projeto(cls, v:str) -> str:
        if not v:
            raise ValueError("Projeto não pode ser vazio")
        return v

    @field_validator("periodo",mode="before") # se semestre for um valor igual a zero ou vazio aparecerá um erro
    @classmethod
    def validate_perido(cls, v:str) -> str:
        if not v:
            raise ValueError("Período não pode ser vazio")
        elif v.strip() == "0":
            raise ValueError("Período não pode ser 0")
        return v

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after") # Vai ocorrer a validação depois que o user for criado
    def validate_user_post(self, v: Any) -> Self: # vai receber o user
        if self.role == Role.Admin and self.name != "Lucas": # Se outra pessoa em vez de Lucas for Admin vai dar um erro
            raise ValueError("Somente Lucas pode ser Admin")

Abaixo temos a função validate que vai receber um dicionário com dados para testar se esses são válidos, sendo good_data, bad_data, bad_role, bad_name, duplicados e dados faltantes.

In [10]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print("User is valid:", user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

def main() -> None:
    test_data = {
        "valid_user": {
            "name": "Lucas",
            "email": "lucas@example.com",
            "password": "StrongPass123",
            "role": "Admin",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": 1
        },
        "invalid_admin": {
            "name": "João",
            "email": "joao@example.com",
            "password": "SecurePass123",
            "role": "Admin",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": 1
        },
        "invalid_email": {
            "name": "Lucas",
            "email": "email_invalido",
            "password": "SecurePass123",
            "role": "Editor",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": 1
        },
        "negative_semestre": {
            "name": "Lucas",
            "email": "lucas@example.com",
            "password": "SecurePass123",
            "role": "Editor",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": -1
        },
        "invalid_role": {
            "name": "Lucas",
            "email": "lucas@example.com",
            "password": "SecurePass123",
            "role": "InvalidRole",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": 1
        },
        "missing_name_password": {
            "email": "lucas@example.com",
            "role": "Editor",
            "projeto": "Fastcamp_Agentes",
            "periodo": "2025.1",
            "semestre": 1
        }
    }

    for example_name, data in test_data.items():
        print(f"Testando {example_name}")
        validate(data)
        print()

Abaixo podemos observar que o good_data é válido, que o bad_role possui um erro na validação do usuário, que o bad_data possui uma senha inválida que não segue as recomendações, o mesmo pode-se dizer do bad_name, o duplicate usa seu nome de usuário na senha e o missing não t6em dado algum.

In [11]:
if __name__ == "__main__":
    main()

Testando valid_user
User is valid: None

Testando invalid_admin
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='João', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

Testando invalid_email
User is invalid:
1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='email_invalido', input_type=str]

Testando negative_semestre
User is invalid:
1 validation error for User
semestre
  Value error, Semestre não pode ser negativo [type=value_error, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

Testando invalid_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=valu

Como pode ser observado no output está sendo realizadas as verificações que foram montadas de maneira personalizada.

# Custom Serilization na prática

In [12]:
import enum
import hashlib
import re
from typing import Any, Self

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

Expressões regulares para validar nomes e senhas

In [13]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

A diferença entre o trecho abaixo e os anteriores é o de que Role agora possui um User que recebe zero e que a class User agora conta com um parâmetro novo em field em password, o parâmetro exclude, ou seja, quando o objeto é serializado a senha não é incluída. Incluise agora temos serializers além dos validators, oferecendo mais controle, como em field_serializer de que quando usado um JSON em role, deve-se retornar o nome para poder armazená-lo em formato JSON, o mesmo vale para o model_serializer que agora conta com o "mode=wrap".

In [14]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=["Lucas_Cristiano"])
    email: EmailStr = Field(
        examples=["lucas_cristiano@discente.ufg.br"],
        description="Email do discente",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Lucas4321"], description="Senha do Usuário"
    )
    role: Role = Field(default=None, description="Ocupação do Usuário")
    projeto: str = Field (examples=["Fastcamp_Agentes"])
    periodo: str = Field (examples=["2025.1"])
    semestre: int = Field (examples=1)

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @field_validator("semestre",mode="before") # se semestre for um valor negativo ou maior que dois aparecerá um erro
    @classmethod
    def validate_semestre(cls, v:int) -> int:
        if v < 0:
            raise ValueError("Semestre não pode ser negativo")
        elif v > 2:
            raise ValueError("Semestre não pode ser maior que 2")

        return v

    @field_validator("projeto",mode="before") # se projeto for vazio irá acontecer um erro
    @classmethod
    def validate_projeto(cls, v:str) -> str:
        if not v:
            raise ValueError("Projeto não pode ser vazio")
        return v

    @field_validator("periodo",mode="before") # se semestre for um valor igual a zero ou vazio aparecerá um erro
    @classmethod
    def validate_perido(cls, v:str) -> str:
        if not v:
            raise ValueError("Período não pode ser vazio")
        elif v.strip() == "0":
            raise ValueError("Período não pode ser 0")
        return v

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after") # Vai ocorrer a validação depois que o user for criado
    def validate_user_post(self, v: Any) -> Self: # vai receber o user
        if self.role == Role.Admin and self.name != "Lucas": # Se outra pessoa em vez de Lucas for Admin vai dar um erro
            raise ValueError("Somente Lucas pode ser Admin")

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name

    @field_serializer("email", when_used="json")
    @classmethod
    def serialize_email(cls, v: str) -> str:
        return v.lower()  # Garante que o email seja serializado em minúsculas

    @field_serializer("semestre", when_used="json")
    @classmethod
    def serialize_semestre(cls, v: int) -> str:
        return f"Semestre {v}" # Retornando semestre de forma padronizada e amigável

    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        """Define os campos a serem serializados"""
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

# Fast API + Serializer

Instalando o FastAPI

In [15]:
%pip install FastAPI

Abaixo temos a importação de datetime para trablhar com datas e horários, Optional (ou seja o campo pode ser None), uuid4 que gera um identificador único aleatório e a própia FastApi. Ao final temos a instância app = FASTAPI para poder adicionar rotas.

In [16]:
from datetime import datetime
from typing import Optional, List # importando List diferente do original
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

A diferença desse trecho abaixo em relação aos anteiores da classe User, reside no fato de que este conta conta com opções extras, como por exemplo a configuração que proíbe a adição de atributos extras, ou seja, não é desejável que a API seja chamada, apenas que sejam adicionados vários atributos ao body. Como anteriormente temos name, email, mas agora esses fazem parte de uma API, além desses há atributos adicionais como uma lista de amigos, um ID, uma data e hora opcional e assim como no exemplo anterior podem ser usados serializadores para especificar como retornar esse objeto na API. Abaixo temos o uso da FastAPI onde é declarado um endpoint, para a obtenção de usuários e outro para a criação de usuários, respectivamente "@app.get" e "@app.post", onde ao receber como argumento User a FastAPI vai garantir que seja feita a validação.

Na função main temos um cliente de teste para executar os testes

In [17]:
class Disciplina(BaseModel):
    codigo: str
    nome: str
    professor: str


class Matricula(BaseModel):
    id: UUID4 = Field(default_factory=uuid4)
    aluno_nome: str
    aluno_email: EmailStr
    disciplina: Disciplina
    periodo: str
    matricula_ts: Optional[datetime] = None

    @field_serializer("matricula_ts")
    def serialize_matricula_ts(self, value: Optional[datetime]):
        """Converte o timestamp da matrícula para o formato ISO 8601."""
        return value.isoformat() if value else None

    @field_serializer("id")
    def serialize_id(self, value: UUID4):
        """Formata o UUID como string."""
        return str(value)


matriculas: List[Matricula] = []


@app.post("/matriculas", response_model=Matricula)
def criar_matricula(matricula: Matricula):
    """Cria uma nova matrícula e define o timestamp."""
    matricula.matricula_ts = datetime.utcnow()
    matriculas.append(matricula)
    return matricula


@app.get("/matriculas", response_model=List[Matricula])
def listar_matriculas():
    """Retorna todas as matrículas."""
    return matriculas


@app.get("/matriculas/{matricula_id}", response_model=Matricula)
def obter_matricula(matricula_id: UUID4):
    """Retorna a matrícula com o ID especificado."""
    matricula = next((m for m in matriculas if m.id == matricula_id), None)
    if matricula:
        return matricula
    return JSONResponse(status_code=404, content={"message": "Matrícula não encontrada"})


def main() -> None:
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                "/matriculas",
                json={
                    "aluno_nome": f"Aluno {i}",
                    "aluno_email": f"aluno{i}@example.com",
                    "disciplina": {"codigo": f"COD{i}", "nome": f"Disciplina {i}", "professor": f"Professor {i}"},
                    "periodo": "2023.2",
                },
            )
            assert response.status_code == 200
            assert response.json()["aluno_nome"] == f"Aluno {i}"
            assert response.json()["id"]

            matricula = Matricula.model_validate(response.json())
            assert str(matricula.id) == response.json()["id"]
            assert matricula.matricula_ts

        response = client.get("/matriculas")
        assert response.status_code == 200
        assert len(response.json()) == 5

        response = client.post(
            "/matriculas",
            json={
                "aluno_nome": "Aluno 5",
                "aluno_email": "aluno5@example.com",
                "disciplina": {"codigo": "COD5", "nome": "Disciplina 5", "professor": "Professor 5"},
                "periodo": "2023.2",
            },
        )
        assert response.status_code == 200
        assert response.json()["aluno_nome"] == "Aluno 5"
        assert response.json()["id"]

        matricula = Matricula.model_validate(response.json())
        assert str(matricula.id) == response.json()["id"]
        assert matricula.matricula_ts

        response = client.get(f"/matriculas/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["aluno_nome"] == "Aluno 5"

        response = client.get(f"/matriculas/{uuid4()}")
        assert response.status_code == 404
        assert response.json()["message"] == "Matrícula não encontrada"


if __name__ == "__main__":
    main()

O código acima usa o FASTAPI para gerenciar matriculas de alunos em matérias da faculdade, ele possui duas classes para funcionar Dicplina e Matricula, que respectivamente contêm (código, nome e professor) e (ID único, nome do aluno, email, disciplina, período e timestamp). A API possuí três rotas com três funções criar matriculas (Post/Matriculas), listar todas as matriculas (Get/matriculas) e obter matricula pelo ID do aluno (Get/matriculas/{matricula_id}). Na função main temos testes para avaliar se a matricula foi realizda.